In [116]:
data_file = "training.1600000.processed.noemoticon.csv"
from csv import reader
# open file in read mode
count = 0
c1 = []
c2 = []
c3 = []

words_to_use = ["love", "hate", "aww", "bad", "good", "great", "fuck", "jesus", "sucks", "suck", "pleased", 
                "fanntastic", "amazing", "enjoy", "excellent", "day", "today", "yesterday", "dad", "mom",
               "kid", "child", "year", "think", "way", "first", "well", "even", "new", "any", "most", "man", "boy",
               "woman", "girl", "time", "person", "sad", "happy", "yes", "no"]

with open(data_file, 'r', encoding='mac_roman') as read_obj:
    # pass the file object to reader() to get the reader object
    csv_reader = reader(read_obj)
    # Iterate over each row in the csv using reader object
    for row in csv_reader:
        st = ([s.strip().lower() for s in row])
        cur_str = []
        found = False
        for w in st[-1].split(' '):
            if len(w) > 0:
                if w in words_to_use:
                    found = True
                    if w[0] not in ['@']:
                        cur_str.append(w.replace("!", "").replace("?", "").replace(".", ""))

        if found:
            if (st[0]) == '0':
                c1.append(" ".join(cur_str))
            if (st[0]) == '2':
                c2.append(" ".join(cur_str))
            if (st[0]) == '4':
                c3.append(" ".join(cur_str))

In [117]:
print(c1[::200])

['day', 'well', 'any any', 'bad', 'new new', 'happy think new', 'no time love', 'man', 'today', 'today', 'time', 'first', 'day', 'well', 'good', 'sad day', 'day', 'no', 'hate', 'bad', 'first', 'no happy', 'good', 'good', 'new', 'girl', 'day', 'most', 'think', 'no', 'no', 'day', 'no', 'sucks', 'love', 'today', 'no way new', 'year', 'time no', 'sad', 'day', 'hate', 'any good', 'way', 'great', 'good', 'love', 'great hate', 'sucks time', 'no sad', 'great day', 'bad', 'new', 'fuck', 'any', 'day new', 'first person', 'way', 'yes yes', 'yesterday', 'well today', 'great', 'time', 'day', 'man today', 'no', 'sad man', 'day', 'no', 'good today', 'love', 'no', 'great', 'think', 'good good day', 'no love', 'no hate', 'aww well', 'aww yesterday', 'way', 'day', 'new love', 'think', 'dad', 'even', 'no', 'amazing', 'new', 'sad day', 'no', 'day', 'fuck hate', 'day today', 'first', 'amazing', 'even', 'think', 'sad time', 'no', 'day', 'think', 'good love', 'no', 'well', 'bad', 'today', 'any any', 'think',

In [118]:
all_c = c1 + c3

In [119]:
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(stop_words={'english'})
X = vectorizer.fit_transform(all_c[::10]).toarray()
train_words = (vectorizer.get_feature_names())
print(X.shape)
print(len(train_words))

(56329, 40)
40


In [120]:
x1 = vectorizer.transform(c1); m1 = x1.mean(axis=0).flatten()
x3 = vectorizer.transform(c3); m3 = x3.mean(axis=0).flatten()
len(m1)
for iw, w in enumerate(words):
    if m1[0, iw] / m3[0, iw] < 2 and m3[0, iw] / m1[0, iw] < 2:
        print(iw, w, m1[0, iw], m3[0, iw])
        
for iw, w in enumerate(words):
    if m1[0, iw] / m3[0, iw] > 2:
        print(iw, w, m1[0, iw], m3[0, iw])

for iw, w in enumerate(words):
    if m3[0, iw] / m1[0, iw] > 2:
        print(iw, w, m1[0, iw], m3[0, iw])

1 any 0.03149982573952997 0.02090439494111619
2 aww 0.01337314376668531 0.007285130753358015
4 boy 0.005831438026149938 0.005632290394576027
5 child 0.0014048627653879056 0.0009615249645569214
6 dad 0.01022926929171225 0.005758622141744079
7 day 0.10235377390685102 0.1114035457109813
9 even 0.04322018978222954 0.022785334287841415
11 first 0.023102986141801927 0.030923204334587166
13 girl 0.011030508157905383 0.012991114667114948
18 kid 0.0007293788781937275 0.0011931331676983748
19 love 0.0028564345229754326 0.0027266602097107255
21 mom 0.016053521319062337 0.011801490714615817
22 most 0.013240202788886449 0.009906514507095064
23 new 0.012068884984496486 0.014521132493927995
24 person 0.054254290939534974 0.09198705801431901
26 suck 0.005996715998548521 0.0058182788001289905
31 way 0.07480984050673561 0.06362558077513052
32 well 0.07528770942693125 0.07969076795664504
33 woman 0.07606379729732247 0.053996294268763345
34 year 0.035836576015292804 0.03536236156146673
35 yesterday 0.0444

In [121]:
test1 = "this is fucking great"
test2 = "this is the most amazing news i've heard in a while, i am so happy"
test3 = "i am so sad this is bad news"

In [124]:
def classify(st, m1, m3, vocabulary):
    words = st.split(' ')
    prob1, prob2 = 1, 1
    for w in words:
        if w in train_words:
            prob1 *= m1[0, vocabulary.index(w)]
            prob2 *= m3[0, vocabulary.index(w)]
            print(m1[0, vocabulary.index(w)], m3[0, vocabulary.index(w)], w)
    return prob1, prob2

In [125]:
classify(test3, m1, m3, train_words)

0.07433556458594082 0.0036495838070774275 sad
0.06442966524024431 0.014415856037954628 bad


(0.004789415541716727, 5.261187476127857e-05)